In [1]:
import sys
sys.path.append('..')
import pytesseract
import easyocr
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt
import numpy as np
from pipeline import Pipeline
from huggingface_hub import hf_hub_download
from tqdm import tqdm
import logging

logging.getLogger("ppocr").setLevel(logging.ERROR)

model_path = hf_hub_download(repo_id="Pikurrot/yolo11n-licenseplates", filename="yolo11n_licenseplates.pt")
pipeline = Pipeline(model_path)

Using device cuda


In [2]:
detections = pipeline.detect("../DatasetTotal/Frontal", 640, 0.25, -1)
crops = pipeline.extract_boxes(detections)


image 1/69 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Frontal/0129KMK.jpg: 288x640 1 License Plate, 64.7ms
image 2/69 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Frontal/0195JHM.jpg: 576x640 1 License Plate, 32.2ms
image 3/69 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Frontal/0225JWL.jpg: 640x480 1 License Plate, 33.9ms
image 4/69 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Frontal/0226MPP.jpg: 640x480 1 License Plate, 8.9ms
image 5/69 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Frontal/0278KXW.jpg: 544x640 1 License Plate, 34.3ms
image 6/69 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Frontal/0304FWK.jpg: 448x640 1 License Plate, 34.6ms
image 7/69 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Frontal/0425KDK_.jpg: 640x480 (no detections), 9.5ms
image 8/69 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Frontal/0492MTT.jpg: 640x640 1 License Plate, 12.1ms
image 9/69 /media/eric/D/repos/ANPR-GIA/notebook

In [7]:
detections2 = pipeline.detect("../DatasetTotal/Above", 640, 0.25, -1)
crops2 = pipeline.extract_boxes(detections2)


image 1/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Above/-------.jpeg: 480x640 1 License Plate, 9.7ms
image 2/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Above/1072FRD.jpg: 640x480 1 License Plate, 9.6ms
image 3/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Above/2953JHW.jpg: 640x480 1 License Plate, 10.5ms
image 4/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Above/2956MJG.jpg: 640x480 1 License Plate, 8.4ms
image 5/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Above/3691MTY.jpg: 640x480 1 License Plate, 8.9ms
image 6/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Above/4373KYG.jpg: 288x640 1 License Plate, 13.7ms
image 7/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Above/6005KFV.jpg: 288x640 2 License Plates, 12.6ms
image 8/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Above/6556KKH.jpg: 640x480 2 License Plates, 8.9ms
image 9/11 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal

In [8]:
detections3 = pipeline.detect("../DatasetTotal/Lateral", 640, 0.25, -1)
crops3 = pipeline.extract_boxes(detections3)


image 1/62 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Lateral/0041LZN.jpg: 288x640 1 License Plate, 15.0ms
image 2/62 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Lateral/0182GLK.jpg: 384x640 1 License Plate, 16.5ms
image 3/62 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Lateral/0192DSP.jpg: 288x640 1 License Plate, 14.4ms
image 4/62 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Lateral/0425KDK.jpg: 640x480 1 License Plate, 9.1ms
image 5/62 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Lateral/0560JDX.jpg: 640x480 1 License Plate, 9.0ms
image 6/62 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Lateral/0568MBX.jpg: 640x480 1 License Plate, 9.8ms
image 7/62 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Lateral/0907JRF.jpg: 384x640 1 License Plate, 16.2ms
image 8/62 /media/eric/D/repos/ANPR-GIA/notebooks/../DatasetTotal/Lateral/1498JBZ.jpg: 384x640 1 License Plate, 13.3ms
image 9/62 /media/eric/D/repos/ANPR-GIA/notebooks/

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
easyocr_reader = easyocr.Reader(['en'])
paddleocr_reader = PaddleOCR(use_angle_cls=True, lang='en')

In [10]:
import time

all_crops = crops + crops2 + crops3

gt = [
	plate["path"].split("/")[-1].split(".")[0]
	for img in all_crops
	for plate in img
]
pred_tesseract = []
pred_easyocr = []
pred_paddleocr = []
time_tesseract = []
time_easyocr = []
time_paddleocr = []

for img in tqdm(all_crops):
	for plate in img:
		path, plate = plate.values()
		start_time = time.time()
		res_tesseract = pytesseract.image_to_data(
			plate,
			config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ',
			output_type=pytesseract.Output.DICT
		)
		res_tesseract = res_tesseract["text"][-1].replace(" ", "")
		time_tesseract.append(time.time() - start_time)
		start_time = time.time()
		res_easyocr = easyocr_reader.readtext(
			np.array(plate),
			allowlist="0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ",
		)
		res_easyocr = "".join([recog[-2] for recog in res_easyocr]).replace(" ", "")
		time_easyocr.append(time.time() - start_time)
		start_time
		res_paddleocr = paddleocr_reader.ocr(np.array(plate), det=False, rec=True, cls=False)
		res_paddleocr = res_paddleocr[0][0][0].replace(" ", "")
		time_paddleocr.append(time.time() - start_time)
		pred_tesseract.append(res_tesseract)
		pred_easyocr.append(res_easyocr)
		pred_paddleocr.append(res_paddleocr)

100%|██████████| 153/153 [00:56<00:00,  2.69it/s]


In [11]:
def accuracy(gt, pred):
	return sum([1 for i in range(len(gt)) if gt[i] == pred[i]]) / len(gt)

from Levenshtein import distance

def compute_similarity_percentage(predictions, gt):
    similarities = []
    for pred, true in zip(predictions, gt):
        d = distance(pred, true)
        max_len = max(len(pred), len(true))
        if max_len > 0:
            similarity = (1 - d / max_len)
        else:
            similarity = 1.0
        similarities.append(similarity)
    
    average_similarity = sum(similarities) / len(similarities) if similarities else 0
    return similarities, average_similarity

print("Tesseract accuracy:", accuracy(gt, pred_tesseract), "Time:", np.mean(time_tesseract))
print("EasyOCR accuracy:", accuracy(gt, pred_easyocr), "Time:", np.mean(time_easyocr))
print("PaddleOCR accuracy:", accuracy(gt, pred_paddleocr), "Time:", np.mean(time_paddleocr))
print("Tesseract similarity:", compute_similarity_percentage(pred_tesseract, gt)[1])
print("EasyOCR similarity:", compute_similarity_percentage(pred_easyocr, gt)[1])
print("PaddleOCR similarity:", compute_similarity_percentage(pred_paddleocr, gt)[1])

Tesseract accuracy: 0.10778443113772455 Time: 0.2082792870298831
EasyOCR accuracy: 0.17365269461077845 Time: 0.09397158651294822
PaddleOCR accuracy: 0.562874251497006 Time: 0.1322282374262096
Tesseract similarity: 0.5790419161676646
EasyOCR similarity: 0.6250951409391882
PaddleOCR similarity: 0.7994368406045052
